In [ ]:
# !git clone https://github.com/EnriqueMejia96/chatbot_qa.git

Cloning into 'chatbot_qa'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 11 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (11/11), 745.27 KiB | 5.65 MiB/s, done.


In [1]:
%cd chatbot_qa

/content/chatbot_qa


In [2]:
!ls -la

total 956
drwxr-xr-x 4 root root   4096 Jan 11 22:14 .
drwxr-xr-x 1 root root   4096 Jan 11 22:03 ..
-rw-r--r-- 1 root root   2832 Jan 11 21:51 app.py
-rw-r--r-- 1 root root     76 Jan 11 22:03 credentials.json
-rw-r--r-- 1 root root   3921 Jan 11 21:51 dmc_logo.jpg
drwxr-xr-x 8 root root   4096 Jan 11 21:51 .git
-rw-r--r-- 1 root root  50192 Jan 11 21:51 Lab_LLM_intro.ipynb
-rw-r--r-- 1 root root 885970 Jan 11 21:51 llm_doc.pdf
drwxr-xr-x 2 root root   4096 Jan 11 22:14 __pycache__
-rw-r--r-- 1 root root     12 Jan 11 21:51 README.md
-rw-r--r-- 1 root root   1822 Jan 11 21:51 utils.py


# 0. Instalación de librerías

In [ ]:
# !pip install streamlit
# !pip install pyngrok==4.1.1
# #https://dashboard.ngrok.com/signup
# !pip install --upgrade typing_extensions
# !pip install openai
# !pip install pypdf
# !pip install langchain

# 1. Crear Vector Store

## 1.1. Carga de documento pdf

In [2]:
# Importe la clase PyPDFLoader del módulo langchain.document_loaders.
from langchain.document_loaders import PyPDFLoader

# Cree una instancia de la clase Py
# PDFLoader, pasando el nombre del archivo del documento PDF que desea cargar.
loader = PyPDFLoader("llm_doc.pdf")

# Este método lee el contenido del archivo PDF especificado al crear la instancia del cargador.
# El contenido del documento cargado se almacena en la variable 'documentos' para su posterior procesamiento.
documents = loader.load()

In [3]:
documents

[Document(page_content='Building Pipelines and Environments for  \nLarge Language Models\nWhitepaper by Dr. Archisman Majumdar, Principal – Mphasis NEXT Labs | Amrit R, Lead Data Scientist - Mphasis NEXT Labs |  \nManami Mandal, Lead Data Scientist - Mphasis NEXT Labs | Dr. Udayaadithya Avadhanam, Principal & VP , Mphasis NEXT Labs |  \nSai Bharath Sundar, Manager (Data Science), Mphasis  NEXT Labs | Biju Mathews, Partner – Mphasis NEXT Labs  |\nSiddharth Shankar, Affiliated to University of Maryland', metadata={'source': 'llm_doc.pdf', 'page': 0}),
 Document(page_content='Contents\nIntroduction to LLMOps 1\nWhy LLMOps? 1\nTypical stages in an LLMOps Workflow 2\n 3.1 Data Collection, Preparation, Labelling 2\n 3.2 Selection of Foundation Models 3\n 3.3 Using Large Language Models - Prompting and Fine-tuning 4\n 3.4 Evaluation of Prompts and Models & Version Control 5\n 3.5 Deployment and Monitoring 6\n 3.6 Security, Privacy, Governance and Ethical Considerations 8\nSetting Up LLMOps Pi

## 1.2. Generación de 'chunks'

In [4]:
# Esta clase se utiliza para dividir textos en fragmentos más pequeños, basándose en el número de caracteres.
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Se especifica el tamaño de cada fragmento de texto (chunk_size) en 1000 caracteres y
# la superposición entre fragmentos consecutivos (chunk_overlap) en 100 caracteres.
text_splitter = RecursiveCharacterTextSplitter(chunk_size    = 1000,
                                               chunk_overlap = 100)

# Este método divide el contenido de los documentos cargados en fragmentos más pequeños,
# basándose en los parámetros de tamaño y superposición especificados.
doc_splits = text_splitter.split_documents(documents)

In [5]:
doc_splits[2].page_content

'1.\nIntroduction to LLMOps\nGenerative AI models have gained wide popularity in recent times with the adoption of  \ntransformer-based neural network architectures. Generative model’s ability to generate new data \nenables them to go beyond traditional prediction and classification use cases. These models \nare now used across domains and use cases like chatbots, question answering, fraud detection, \nprotein folding and many more.\nGenerative AI models for natural language use cases are powered by Large Language Models (LLMs). \nLLMs are transformer-based Deep Learning architectures that harness vast amounts of textual \ndata to develop language and domain understanding. The models are built with an emphasis on \ngenerating human-like responses and reasoning. Their ability to understand human languages allows \nthem to serve as powerful tools for information retrieval, natural language processing, language \ntranslation and even creative writing.'

In [6]:
doc_splits[3].page_content

'translation and even creative writing.\nUsing large language models in production environments poses a certain unique set of challenges \nsuch as organizing LLMs into agents for sub-tasks, developing robust instructions for each  \nLLM agent, evaluating the correctness of generated response and efficiencies with fine-tuning.  \nHence, effective usage in a production environment requires appropriate infrastructure and practices \nfocusing on experimentation, deployment, management and monitoring of large language models. \nLarge Language Model Operations (LLMOps) is a framework of tools and best practices  \nto manage the lifecycle of LLM-powered applications, from development to deployment  \nand maintenance.\nThe aim is to enable AI capabilities with LLMs by developing better prompts, longer context, \nfaster inference and customized techniques that enable rapid experimentation and innovation \nwith LLMs. Together, these allow data scientists and engineers to collaborate effectively 

In [7]:
doc_splits[3].metadata

{'source': 'llm_doc.pdf', 'page': 2}

## 1.3. Crear Vector Store como dataframe

In [8]:
import pandas as pd
# Crear una lista de diccionarios a partir de los fragmentos de documento divididos previamente.
# Cada diccionario contiene dos claves: 'Chunks', que almacena el contenido del fragmento de texto,
# y 'Metadata', que almacena los metadatos asociados a dicho fragmento.
data = [{'Chunks': doc.page_content, 'Metadata': doc.metadata} for doc in doc_splits]
# Crear un DataFrame de pandas a partir de la lista de diccionarios 'data'.
df_vector_store = pd.DataFrame(data)
df_vector_store.head()

,Chunks,Metadata
0,Building Pipelines and Environments for \nLar...,"{'source': 'llm_doc.pdf', 'page': 0}"
1,Contents\nIntroduction to LLMOps 1\nWhy LLMOps...,"{'source': 'llm_doc.pdf', 'page': 1}"
2,1.\nIntroduction to LLMOps\nGenerative AI mode...,"{'source': 'llm_doc.pdf', 'page': 2}"
3,translation and even creative writing.\nUsing ...,"{'source': 'llm_doc.pdf', 'page': 2}"
4,"with LLMs. Together, these allow data scientis...","{'source': 'llm_doc.pdf', 'page': 2}"


In [9]:
# Importar la clase OpenAI del módulo openai. Esta clase proporciona acceso a las API de OpenAI.
from openai import OpenAI
import numpy as np
import json
# Abrir el archivo 'credentials.json' en modo de lectura.
file_name = open('credentials.json')
# Cargar el contenido del archivo JSON en una variable llamada 'config_env'.
config_env = json.load(file_name)
# Extraer el valor asociado a la clave 'openai_key' del diccionario 'config_env'.
api_key = config_env["openai_key"]
# Crear una instancia de la clase OpenAI, proporcionando la clave API extraída del archivo de configuración.
client = OpenAI(api_key=api_key)

# Llamar al método 'embeddings.create' del cliente de OpenAI para crear embeddings del texto.
# Se especifica el modelo 'text-embedding-ada-002', el texto de entrada y el formato de codificación.
# La función retorna la representación vectorial del texto.
def text_embedding(text=[]):
    embeddings = client.embeddings.create(model="text-embedding-ada-002",
                                          input=text,
                                          encoding_format="float")
    return embeddings.data[0].embedding

# Añadir una nueva columna 'Embedding' al DataFrame 'df_vector_store'.
df_vector_store["Embedding"] = df_vector_store["Chunks"].apply(lambda x: text_embedding([x]))
df_vector_store["Embedding"] = df_vector_store["Embedding"].apply(np.array)

# Guardar el DataFrame 'df_vector_store' en un archivo con formato pickle.
df_vector_store.to_pickle('df_vector_store.pkl')
df_vector_store.head()

,Chunks,Metadata,Embedding
0,Building Pipelines and Environments for \nLar...,"{'source': 'llm_doc.pdf', 'page': 0}","[-0.00042193758, -0.012498714, 0.017564613, -0..."
1,Contents\nIntroduction to LLMOps 1\nWhy LLMOps...,"{'source': 'llm_doc.pdf', 'page': 1}","[0.0030914678, -0.0044048806, -0.002168892, -0..."
2,1.\nIntroduction to LLMOps\nGenerative AI mode...,"{'source': 'llm_doc.pdf', 'page': 2}","[-0.021246167, -0.008550943, -0.013926398, -0...."
3,translation and even creative writing.\nUsing ...,"{'source': 'llm_doc.pdf', 'page': 2}","[-0.006791799, -0.0054946076, -0.0034398423, -..."
4,"with LLMs. Together, these allow data scientis...","{'source': 'llm_doc.pdf', 'page': 2}","[0.007078849, -0.010141378, 0.003159643, -0.05..."


# 2.  Formulación de pregunta

In [10]:
# Definir una cadena de texto que contiene la consulta de interés.
query = '¿Cómo se selecciona un modelo llm?'
# Generar la representación vectorial (embedding) de la consulta de texto utilizando la función 'text_embedding'.
query_embedding = text_embedding(query)
query_embedding

[-0.02419639,
 0.011456056,
 0.0020118116,
 -0.02696989,
 -0.041670803,
 0.028281495,
 -0.026204787,
 0.024128078,
 -0.010930046,
 -0.023649888,
 0.017911615,
 0.025945198,
 -0.012064039,
 0.0051200436,
 -0.02553532,
 -0.0005460754,
 -0.0007160035,
 -0.0006946557,
 0.026723964,
 -0.009092432,
 -0.0063974927,
 -0.012945274,
 0.028008245,
 -0.031314585,
 -0.0047614016,
 0.0010187145,
 0.019687746,
 -0.013792353,
 0.0022013797,
 -0.001734291,
 0.021477541,
 0.024059765,
 -0.026997216,
 -0.019400833,
 -0.019428158,
 0.00843663,
 0.01617647,
 -0.003828932,
 0.018376142,
 -0.0077535016,
 0.024606267,
 0.01409976,
 0.014946839,
 -0.0071113617,
 0.021545855,
 -0.005857822,
 -0.017091861,
 0.0025002481,
 -0.03391047,
 -0.0010246919,
 -0.019455483,
 0.014946839,
 -0.020439187,
 -0.007828646,
 0.0052771633,
 0.0039860513,
 -0.0049902494,
 -0.004006545,
 0.021819105,
 -0.012863299,
 -0.0022936019,
 -0.012098195,
 -0.0121118575,
 0.016804947,
 0.006424818,
 0.00070831826,
 0.0005144807,
 0.00100846

# 3. Búsqueda semántica

In [11]:
# Definir una función para calcular el producto punto entre dos vectores.
def get_dot_product(row):
    return np.dot(row, query_vector)

# Definir una función para calcular la similitud de coseno entre dos vectores.
def cosine_similarity(row):
    denominator1 = np.linalg.norm(row)
    denominator2 = np.linalg.norm(query_vector.ravel())
    dot_prod = np.dot(row, query_vector)
    return dot_prod/(denominator1*denominator2)

# Definir una función para obtener los fragmentos de texto más relevantes desde un almacenamiento
# vectorial dada una consulta.
def get_context_from_query(query, vector_store, n_chunks = 5):
    global query_vector
    # Convertir el embedding de la consulta en un array de numpy.
    query_vector = np.array(query_embedding)
    # Calcular la similitud de coseno para cada vector en el almacenamiento y ordenar los resultados
    # de mayor a menor similitud, seleccionando los índices de los 'n_chunks' más altos.
    top_matched = (
        vector_store["Embedding"]
        .apply(cosine_similarity)
        .sort_values(ascending=False)[:n_chunks]
        .index)
    # Seleccionar los fragmentos de texto correspondientes a los índices de mayor similitud.
    top_matched_df = vector_store[vector_store.index.isin(top_matched)][["Chunks"]]
    # Devolver una lista con los fragmentos de texto seleccionados.
    return list(top_matched_df['Chunks'])

# Utilizar la función 'get_context_from_query' para obtener los fragmentos de texto más relevantes
# dada la consulta especificada y el almacenamiento vectorial 'df_vector_store', limitando el número
# de fragmentos a 5.
Context_List = get_context_from_query(
    query        = query,
    vector_store = df_vector_store,
    n_chunks     = 5)

# Iterar sobre la lista de fragmentos de texto relevantes e imprimir cada uno, separados por líneas
for chunk in Context_List:
  print("#########################")
  print(chunk)

#########################
Contents
Introduction to LLMOps 1
Why LLMOps? 1
Typical stages in an LLMOps Workflow 2
 3.1 Data Collection, Preparation, Labelling 2
 3.2 Selection of Foundation Models 3
 3.3 Using Large Language Models - Prompting and Fine-tuning 4
 3.4 Evaluation of Prompts and Models & Version Control 5
 3.5 Deployment and Monitoring 6
 3.6 Security, Privacy, Governance and Ethical Considerations 8
Setting Up LLMOps Pipelines 9
Conclusions 11
References 11
#########################
translation and even creative writing.
Using large language models in production environments poses a certain unique set of challenges 
such as organizing LLMs into agents for sub-tasks, developing robust instructions for each  
LLM agent, evaluating the correctness of generated response and efficiencies with fine-tuning.  
Hence, effective usage in a production environment requires appropriate infrastructure and practices 
focusing on experimentation, deployment, management and monitoring of l

# 5. Construir prompt

In [12]:
custom_prompt = """
Eres una Inteligencia Artificial super avanzada que trabaja asistente personal.
Utilice los RESULTADOS DE BÚSQUEDA SEMANTICA para responder las preguntas del usuario.
Solo debes utilizar la informacion de la BUSQUEDA SEMANTICA si es que hace sentido y tiene relacion con la pregunta del usuario.
Si la respuesta no se encuentra dentro del contexto de la búsqueda semántica, no inventes una respuesta, y responde amablemente que no tienes información para responder.

RESULTADOS DE BÚSQUEDA SEMANTICA:
{source}

Lee cuidadosamente las instrucciones, respira profundo y escribe una respuesta para el usuario!
""".format(source = str(Context_List))

# 6. Obtener respuesta

In [14]:
from openai import OpenAI
import json
# Abrir el archivo 'credentials.json' en modo de lectura.
file_name = open('credentials.json')
# Cargar el contenido del archivo JSON en una variable llamada 'config_env'.
config_env = json.load(file_name)
# Extraer el valor asociado a la clave 'openai_key' del diccionario 'config_env'.
api_key = config_env["openai_key"]
# Crear una instancia de la clase OpenAI, proporcionando la clave API extraída del archivo de configuración.
client = OpenAI(api_key=api_key)

# Utilizar el cliente de OpenAI para crear una completación de chat mediante el modelo "gpt-4".
# Se especifican varios parámetros en la llamada, incluyendo:
# - model: el identificador del modelo de lenguaje a utilizar, en este caso "gpt-4".
# - temperature: controla la aleatoriedad de las respuestas. Un valor de 0.0 genera la respuesta más probable.
# - messages: una lista de mensajes que conforman el contexto de la conversación. Se incluyen dos mensajes:
#   1. Un mensaje de rol "system", que proporciona instrucciones o contexto al modelo.
#   2. Un mensaje de rol "user", que contiene la consulta del usuario.
# 'custom_prompt': Variable definida previamente que contiene el texto del prompt del sistema.
completion = client.chat.completions.create(
  model="gpt-4",
  temperature = 0.0,
  messages=[
    {"role": "system", "content": custom_prompt},
    {"role": "user", "content": query}
  ]
)

# Imprimir el contenido del mensaje de la primera opción de completación generada por el modelo.
print(completion.choices[0].message.content)

La selección de un modelo de Lenguaje de Gran Escala (LLM, por sus siglas en inglés) implica varios factores. Dado que entrenar LLMs desde cero es costoso y consume mucho tiempo, muchos investigadores y profesionales optan por utilizar LLMs preentrenados como modelos base, que pueden ser ajustados o adaptados a aplicaciones específicas con menos datos y recursos.

Algunos modelos base populares incluyen BERT, GPT-3, T5 y XLNet. Estos modelos han demostrado ser efectivos para una amplia gama de tareas, incluyendo la comprensión del lenguaje natural, la generación de lenguaje natural y la traducción automática.

Al seleccionar un modelo base para un proyecto LLM, es importante considerar varios factores:

1. Rendimiento del modelo en tareas de referencia.
2. Disponibilidad para ejecutarse en hardware menos costoso.
3. Disponibilidad de modelos preentrenados específicos del dominio. Estos son modelos base que están especializados para tareas que pueden ser seleccionadas según los casos de

# 7. Ejecutar Streamlit App

In [24]:
from google.colab import userdata
from pyngrok import ngrok
ngrok_token = userdata.get('ngrok_token')
!ngrok authtoken $ngrok_token

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [26]:
!nohup streamlit run app.py &
public_url = ngrok.connect(port='8501')
public_url

nohup: appending output to 'nohup.out'


'http://6f19-34-68-3-131.ngrok-free.app'

In [25]:
# Terminate all active ngrok tunnels
ngrok.kill()